## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Jamestown,US,2022-02-06 17:18:41,42.0970,-79.2353,31.51,40,0,8.01
1,1,Islahiye,TR,2022-02-06 17:22:22,37.0250,36.6306,42.58,89,97,3.74
2,2,Olinda,BR,2022-02-06 17:22:22,-8.0089,-34.8553,85.73,58,20,9.22
3,3,Meybod,IR,2022-02-06 17:22:22,32.2501,54.0166,48.22,21,0,2.26
4,4,Nara,JP,2022-02-06 17:22:22,34.6851,135.8049,38.48,62,12,4.63


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = '' # abusing the falseness of a blank string to loop until both are valid
max_temp = ''
while not (min_temp and max_temp):
    try:
        min_temp = float(input('enter minimum temperature (F): ')) if not min_temp else min_temp
        max_temp = float(input('enter maximum temperature (F): ')) if not max_temp else max_temp
        if (max_temp <= min_temp and (max_temp and min_temp)):
            print('max temperature must be greater than min')
            max_temp = '' # abusing that empty string again :~)
    except (ValueError):
        print('please enter a number')

enter minimum temperature (F): 30
enter maximum temperature (F): 40


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_data_df = city_data_df.loc[city_data_df['Max Temp'] > min_temp]
city_data_df = city_data_df.loc[city_data_df['Max Temp'] < max_temp]
city_data_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Jamestown,US,2022-02-06 17:18:41,42.0970,-79.2353,31.51,40,0,8.01
4,4,Nara,JP,2022-02-06 17:22:22,34.6851,135.8049,38.48,62,12,4.63
11,11,Zharkovskiy,RU,2022-02-06 17:22:24,55.8490,32.2697,30.11,90,100,9.64
19,19,Klaksvik,FO,2022-02-06 17:12:28,62.2266,-6.5890,36.52,61,87,37.51
21,21,Brae,GB,2022-02-06 17:22:25,60.3964,-1.3530,39.11,59,100,30.40
...,...,...,...,...,...,...,...,...,...,...
1701,1701,Jamestown,US,2022-02-06 18:02:20,42.0970,-79.2353,33.53,36,0,5.01
1725,1725,Tal,PK,2022-02-06 18:06:16,35.4791,72.2439,31.51,97,100,3.98
1737,1737,Jamestown,US,2022-02-06 18:02:20,42.0970,-79.2353,33.53,36,0,5.01
1746,1746,Brigantine,US,2022-02-06 18:06:20,39.4101,-74.3646,37.99,59,0,9.22


In [5]:
# 4a. Determine if there are any empty rows.
# no need to check, just do it unconditionally

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = city_data_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df.copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
0,0,Jamestown,US,2022-02-06 17:18:41,42.0970,-79.2353,31.51,40,0,8.01,
4,4,Nara,JP,2022-02-06 17:22:22,34.6851,135.8049,38.48,62,12,4.63,
11,11,Zharkovskiy,RU,2022-02-06 17:22:24,55.8490,32.2697,30.11,90,100,9.64,
19,19,Klaksvik,FO,2022-02-06 17:12:28,62.2266,-6.5890,36.52,61,87,37.51,
21,21,Brae,GB,2022-02-06 17:22:25,60.3964,-1.3530,39.11,59,100,30.40,
25,25,Gravdal,NO,2022-02-06 17:22:26,68.1214,13.5031,33.69,64,40,5.75,
37,37,Krasnoye,UA,2022-02-06 17:22:29,49.9145,24.6134,36.75,77,100,16.04,
42,42,Jamestown,US,2022-02-06 17:18:41,42.0970,-79.2353,31.51,40,0,8.01,
43,43,Sinop,TR,2022-02-06 17:22:30,41.7711,34.8709,36.03,95,98,2.75,
46,46,Jamestown,US,2022-02-06 17:18:41,42.0970,-79.2353,31.51,40,0,8.01,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
payload = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    "location": "48.8566,2.3522"
}
url_base = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotel_names = []

print('starting hotel retrieval')
# 6b. Iterate through the hotel DataFrame.
for idx in hotel_df.index:
    # 6c. Get latitude and longitude from DataFrame
    lat = hotel_df['Lat'][idx]
    lng = hotel_df['Lng'][idx]
    payload["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    url = url_base

    # 6e. Make request and retrieve the JSON data from the search. 
    response = requests.get(url, params=payload)
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.

    try:
        hotel_df['Hotel Name'][idx] = response.json()['results'][0]['name']
    except (IndexError):
        hotel_df['Hotel Name'][idx] = None
        
print('hotels retrieved')

starting hotel retrieval


C:\Users\schap\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\schap\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


hotels retrieved


In [23]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()
hotel_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
0,0,Jamestown,US,2022-02-06 17:18:41,42.0970,-79.2353,31.51,40,0,8.01,Hampton Inn & Suites Jamestown
4,4,Nara,JP,2022-02-06 17:22:22,34.6851,135.8049,38.48,62,12,4.63,Guesthouse Nara Komachi
11,11,Zharkovskiy,RU,2022-02-06 17:22:24,55.8490,32.2697,30.11,90,100,9.64,"Gostinitsa ""Mezha"""
19,19,Klaksvik,FO,2022-02-06 17:12:28,62.2266,-6.5890,36.52,61,87,37.51,Airbnb rooms
21,21,Brae,GB,2022-02-06 17:22:25,60.3964,-1.3530,39.11,59,100,30.40,Busta House Hotel
25,25,Gravdal,NO,2022-02-06 17:22:26,68.1214,13.5031,33.69,64,40,5.75,Skreda Rorbusuiter
37,37,Krasnoye,UA,2022-02-06 17:22:29,49.9145,24.6134,36.75,77,100,16.04,Feniks
42,42,Jamestown,US,2022-02-06 17:18:41,42.0970,-79.2353,31.51,40,0,8.01,Hampton Inn & Suites Jamestown
46,46,Jamestown,US,2022-02-06 17:18:41,42.0970,-79.2353,31.51,40,0,8.01,Hampton Inn & Suites Jamestown
104,104,Jamestown,US,2022-02-06 17:18:41,42.0970,-79.2353,31.51,40,0,8.01,Hampton Inn & Suites Jamestown


In [14]:
# 8a. Create the output File (CSV)
output_file = './outputs/hotels.csv'
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_file, index_label="City_ID")

In [30]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [33]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))